**stackUserViz**
This note book allows the collection and visualisation of user data from Stack Exchange network sites. It collects timestamps for posting activity events and for reputation score events, and then plots those events on a timeline.

In [ ]:
# Collect posting activity and reputation score event data for one or more Stack Exchange user. Specifiy the particular site of interest e.g. stackoverflow

from stackapi import StackAPI
import pandas as pd

def collect_user_ids():
    user_ids = input("Please enter user ids, separated by a comma: ")
    return [user_id.strip() for user_id in user_ids.split(',')]

def collect_reputation_events(user_id, site):
    events = site.fetch(f'users/{user_id}/reputation-history')
    return pd.DataFrame(events['items'])

def process_reputation_events(df):
    df = df.iloc[::-1]
    df['time_delta'] = df['creation_date'].diff()
    df['days'] = df['time_delta'].cumsum() / 86400
    df['reputation'] = df['reputation_change'].cumsum()
    df['creation_date'] = pd.to_datetime(df['creation_date'], unit='s')
    return df

def write_reputation_trajectory_to_csv(df, user_id):
    df.to_csv(f'reputation_trajectory_{user_id}.csv')

def main():
    community = input("Please enter the community name: ").strip()
    api_key = 'ADD STACK EXCHANGE API KEY HERE'
    site = StackAPI(community, key=api_key)
    site.page_size = 100
    site.max_pages = 1000

    user_ids = collect_user_ids()

    for i, user_id in enumerate(user_ids):
        events_df = collect_reputation_events(user_id, site)
        processed_df = process_reputation_events(events_df)
        write_reputation_trajectory_to_csv(processed_df, user_id)
        print(f'Reputation trajectories collected: {i + 1}')

    print('All done')

if __name__ == "__main__":
    main()

In [ ]:
# Import csv files with Stack Exchange posting activity and reputation score event data and visualize it

import pandas as pd
import plotly.express as px
import os
import tkinter as tk
from tkinter import filedialog, messagebox

# Create a function to ask the user to select a csv file
def select_csv_file(instruction):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    messagebox.showinfo("Instructions", instruction)
    file_path = filedialog.askopenfilename(filetypes=[('CSV Files', '*.csv')])
    root.destroy()
    return file_path

# Prompt the user to select a csv file for each data frame
df_file = select_csv_file("Please select the csv file for the main posting data (community_userId.csv).")
repdf_file = select_csv_file("Please select the csv file for the reputation data (community_userId_reputation.csv).")

# Import user data and visualize it
df = pd.read_csv(df_file, dtype='object', parse_dates=['creation_date'])
repdf = pd.read_csv(repdf_file, dtype='object', parse_dates=['creation_date'])

# Process main posting data
df['month_year'] = pd.to_datetime(df['creation_date']).dt.to_period('M').dt.to_timestamp()
df1 = df.groupby(['month_year', 'post_type'])['creation_date'].count().unstack(fill_value=0)
df1 = df1.resample('MS').asfreq().fillna(0).astype(int)
df1.columns = ['Answers', 'Comments', 'Questions']

# Process reputation data
repdf1 = repdf[['creation_date', 'reputation']].copy()
repdf1['month_year'] = pd.to_datetime(repdf1['creation_date']).dt.to_period('M').dt.to_timestamp()
repdf1 = repdf1.groupby('month_year')['reputation'].max().reset_index()
repdf1['Reputation'] = repdf1['reputation'].astype(int)

# Create a merged dataframe, but ignore meta if empty
merged = pd.merge(df1, repdf1, on=['month_year'], how='outer')

# Plot the figures using Plotly
fig = px.bar(merged, x='month_year', y=['Answers', 'Comments', 'Questions'], color_discrete_map={'Answers': '#284E60', 'Comments': '#F99B45', 'Questions': '#63AAC0'})
fig.add_scatter(x=merged['month_year'], y=merged['Reputation'], name='Reputation', mode='lines', yaxis='y2', line=dict(color='#D95980', width=5))

fig.update_layout(
    barmode='stack',
    xaxis=dict(tickformat='%Y-%m', title='Month'),
    yaxis=dict(title='Posts'),
    yaxis2=dict(title='Reputation Score', overlaying='y', side='right'),
    title=dict(text=f'{community} participation trajectory', x=0.06),
    legend=dict(x=0, y=1.1, orientation='h', itemsizing='constant', title=""),
)

fig.show()